In [2]:
import dask.dataframe as dd
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from pathlib import Path

In [3]:
base_dir = Path.home()
manipulated_data_directory = base_dir / "NaMinhaRua" / "Ficheiros de dados Manipulados"
df = pd.read_csv(manipulated_data_directory/ 'GOPI&IPMA_MERGED_24h_final.csv')
df.head(2)



,dt_registo,Subseccao,Freguesia,BNFDRB,CA,CDPC2RD,CDRD,CDEEP,DDP,EOVRDJOPANVP,...,RRPDR,SOOLA,ÁAORM,Fim_de_Semana,Feriado,Inverno,Primavera,Verao,Outono,Chuva
0,2018-01-01,11066601201,Santo António,False,False,False,False,False,False,True,...,False,False,False,False,True,True,False,False,False,NaN
1,2018-01-01,11060200605,Alcântara,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,NaN


In [4]:
dia_max_precip = df.loc[df['Chuva'].idxmax()]

print(dia_max_precip)

dt_registo          2020-10-19
Subseccao          11066600601
Freguesia        Santo António
BNFDRB                   False
CA                        True
CDPC2RD                  False
CDRD                     False
CDEEP                    False
DDP                      False
EOVRDJOPANVP              True
EA                       False
FDDDCDPC2ROSM            False
G                         True
LDVPDDPVELDVP            False
OIEVPER                  False
PED                      False
RNADRDDRSU               False
RSRPDP                    True
RJPDR                    False
RMPDR                     True
RRPDR                    False
SOOLA                    False
ÁAORM                    False
Fim_de_Semana            False
Feriado                  False
Inverno                  False
Primavera                False
Verao                    False
Outono                    True
Chuva                48.416667
Name: 45695, dtype: object


In [5]:
unique_subseccao_count = df['Subseccao'].nunique()

print(unique_subseccao_count)

344


In [6]:
num_linhas = df.shape[0]

print(f"Número de linhas: {num_linhas}")
print(df.dtypes)

Número de linhas: 102081
dt_registo        object
Subseccao          int64
Freguesia         object
BNFDRB              bool
CA                  bool
CDPC2RD             bool
CDRD                bool
CDEEP               bool
DDP                 bool
EOVRDJOPANVP        bool
EA                  bool
FDDDCDPC2ROSM       bool
G                   bool
LDVPDDPVELDVP       bool
OIEVPER             bool
PED                 bool
RNADRDDRSU          bool
RSRPDP              bool
RJPDR               bool
RMPDR               bool
RRPDR               bool
SOOLA               bool
ÁAORM               bool
Fim_de_Semana       bool
Feriado             bool
Inverno             bool
Primavera           bool
Verao               bool
Outono              bool
Chuva            float64
dtype: object


In [7]:
num_nan_chuva = df['Chuva'].isna().sum()
print(f"Número de valores NaN na coluna 'Chuva': {num_nan_chuva}")

# Remover as linhas onde 'Chuva' é NaN
df.dropna(subset=['Chuva'], inplace=True)

Número de valores NaN na coluna 'Chuva': 19324


In [8]:
num_linhas = df.shape[0]

print(f"Número de linhas: {num_linhas}")

Número de linhas: 82757


In [9]:
max_chuva = df['Chuva'].max()

# Exibir o resultado
print(f"O valor máximo para 'Chuva' é: {max_chuva}")

O valor máximo para 'Chuva' é: 48.41666666666666


In [10]:
df.head(2)

,dt_registo,Subseccao,Freguesia,BNFDRB,CA,CDPC2RD,CDRD,CDEEP,DDP,EOVRDJOPANVP,...,RRPDR,SOOLA,ÁAORM,Fim_de_Semana,Feriado,Inverno,Primavera,Verao,Outono,Chuva
15290,2018-12-17,11066600603,Santo António,True,False,False,False,False,True,True,...,False,False,False,False,False,True,False,False,False,0.0
15291,2018-12-17,11063301201,Olivais,False,False,False,False,False,False,True,...,False,False,True,False,False,True,False,False,False,0.0


In [4]:
#Uncomment to analyze by precipitation volume
# df['Chuva'] > 10 cria uma série booleana onde cada valor é True se for maior que 10, e False caso contrário
#Aguaceiros fortes
#df['Chuva'] = df['Chuva'].apply(lambda x: True if x > 10 else False)
#Aguaceiros moderados
#df['Chuva'] = df['Chuva'].apply(lambda x: True if 2 <= x <= 10 else False)
# Aguaceiros fracos
df['Chuva'] = df['Chuva'].apply(lambda x: True if x!=0 and x <= 2 else False)
# Valor exploratório da regra
#df['Chuva'] = df['Chuva'].apply(lambda x: True if 0 < x < 5 else False)
# Impacto quando há chuva
#df['Chuva'] = df['Chuva'].apply(lambda x: True if x > 0 else False)

for col in df.select_dtypes(include='number').columns:
           df[col] = df[col].astype(bool)

# Define relevant columns for Apriori analysis
relevant_columns = ['BNFDRB', 'CA', 'CDPC2RD', 'CDRD', 'CDEEP', 'DDP', 
                    'EOVRDJOPANVP', 'EA', 'FDDDCDPC2ROSM', 'G', 'LDVPDDPVELDVP', 
                    'OIEVPER', 'PED', 'RNADRDDRSU', 'RSRPDP', 'RJPDR', 
                    'RMPDR', 'RRPDR', 'SOOLA', 'ÁAORM', 'Fim_de_Semana', 'Feriado','Chuva','Inverno','Outono','Primavera','Verao']

# Keep only the relevant columns in the basket
basket = df[relevant_columns]

# Optimize data types by converting boolean columns to int8 (takes less memory)
basket = basket.astype('int8')

# Apply fpgrowth algorithm with minimum support
start_time = time.time()
frequent_itemsets = fpgrowth(basket, min_support=0.01, use_colnames=True)
#0.0027
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
fp_growth_exec_time = time.time() - start_time
print("Itemsets frequentes:\n", frequent_itemsets.head(30))



C:\Users\jmvdp\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Itemsets frequentes:
        support                           itemsets
0     0.937323                            (RMPDR)
1     0.726718                     (EOVRDJOPANVP)
25    0.692256              (RMPDR, EOVRDJOPANVP)
2     0.484624                            (RJPDR)
26    0.477736                     (RMPDR, RJPDR)
9     0.406153                       (RNADRDDRSU)
5191  0.398673                (RNADRDDRSU, RMPDR)
27    0.386916              (EOVRDJOPANVP, RJPDR)
28    0.382312       (RJPDR, RMPDR, EOVRDJOPANVP)
19    0.381672                              (PED)
9678  0.370833                       (RMPDR, PED)
12    0.340273                    (LDVPDDPVELDVP)
10    0.337579                            (ÁAORM)
13    0.337349                               (CA)
14    0.332842                            (SOOLA)
5866  0.328951             (RMPDR, LDVPDDPVELDVP)
3     0.328589                           (RSRPDP)
5198  0.326075                     (ÁAORM, RMPDR)
5192  0.325145         (RNAD

In [5]:
# Generate association rules with lift metric and minimum threshold
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=0.01)
rules = rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])
print("Regras de associação:\n", rules.head(50))

Regras de associação:
                                               antecedents  \
381816               (CA, SOOLA, Primavera, EOVRDJOPANVP)   
381817                              (ÁAORM, RMPDR, CDEEP)   
381820                                 (CA, RMPDR, CDEEP)   
381813            (Primavera, SOOLA, ÁAORM, EOVRDJOPANVP)   
381650               (CA, SOOLA, Primavera, EOVRDJOPANVP)   
381780        (RMPDR, SOOLA, Primavera, CA, EOVRDJOPANVP)   
381671                                     (ÁAORM, CDEEP)   
381853                                     (ÁAORM, CDEEP)   
381785                (ÁAORM, RMPDR, EOVRDJOPANVP, CDEEP)   
381848                             (CA, SOOLA, Primavera)   
381808                      (CA, Primavera, SOOLA, RMPDR)   
381825                       (ÁAORM, EOVRDJOPANVP, CDEEP)   
381842                          (Primavera, SOOLA, ÁAORM)   
381791                   (CA, RMPDR, EOVRDJOPANVP, CDEEP)   
381802                   (Primavera, ÁAORM, RMPDR, SOOLA)   
3

In [12]:
# Define relevant columns for Apriori analysis
relevant_columns = ['BNFDRB', 'CA', 'CDPC2RD', 'CDRD', 'CDEEP', 'DDP', 
                    'EOVRDJOPANVP', 'EA', 'FDDDCDPC2ROSM', 'G', 'LDVPDDPVELDVP', 
                    'OIEVPER', 'PED', 'RNADRDDRSU', 'RSRPDP', 'RJPDR', 
                    'RMPDR', 'RRPDR', 'SOOLA', 'ÁAORM', 'Fim_de_Semana', 'Feriado','Chuva','Inverno','Outono','Primavera','Verao']

# Keep only the relevant columns in the basket
basket = df[relevant_columns]
basket_sample = basket.sample(frac=0.3, random_state=42)

# Apply fpgrowth algorithm with minimum support
start_time = time.time()
frequent_itemsets = fpgrowth(basket_sample, min_support=0.0027, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
fp_growth_exec_time = time.time() - start_time
print("FP-Growth execution time:", fp_growth_exec_time)
print("Itemsets frequentes:\n", frequent_itemsets.head(30))

# Apply apriori algorithm with minimum support
start_time = time.time()
frequent_itemsets_apriori = apriori(basket_sample, min_support=0.0027, use_colnames=True, low_memory=True)
frequent_itemsets_apriori = frequent_itemsets_apriori.sort_values(by='support', ascending=False)
apriori_exec_time = time.time() - start_time
print("Apriori execution time:", apriori_exec_time)
print("Itemsets frequentes Apriori:\n", frequent_itemsets_apriori.head(30))

# Function to compare execution times
def compare_time_exec(algo1, algo2):
    execution_times = [algo1[1], algo2[1]]
    algo_names = [algo1[0], algo2[0]]
    y = np.arange(len(algo_names))
    
    plt.bar(y, execution_times, color=['grey', 'black'])
    plt.xticks(y, algo_names)
    plt.xlabel('Algoritmos')
    plt.ylabel('Tempo (segundos)')
    plt.title("Tempo de execução (segundos) comparativo")
    plt.show()

# Prepare the algorithm execution time data for comparison
algo1 = ['Fp Growth', fp_growth_exec_time]
algo2 = ['Apriori', apriori_exec_time]


# Call the comparison function
compare_time_exec(algo1, algo2)


C:\Users\jmvdp\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 0.0375000008692344

In [ ]:
#df = pd.read_csv("Merge_Gopi_With_Ipma_Precacumulada_Estacao.csv")
df.head(2)
chuva_frequency = df['Chuva'].max()

# Print the frequency
print(chuva_frequency)

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
# Plot lift vs confidence
plt.figure(figsize=(10, 6))
sns.scatterplot(data=rules, x='lift', y='confidence', hue='support', size='support', sizes=(20, 200))
plt.title('Lift vs Confidence')
plt.xlabel('Lift')
plt.ylabel('Confidence')
plt.legend(loc='best')
plt.show()

NameError: name 'rules' is not defined

<Figure size 1000x600 with 0 Axes>

In [6]:
# Filter rules involving 'Fim_de_Semana'
rules_pressão_acumulada = rules[(rules['antecedents'].apply(lambda x: 'Chuva' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Chuva' in str(x)))]

print("rules_pressão_acumulada 'pressão_acumulada':\n", rules_pressão_acumulada.head(30))



rules_pressão_acumulada 'pressão_acumulada':
                                       antecedents  \
269397        (SOOLA, RMPDR, EOVRDJOPANVP, Chuva)   
269424                       (RNADRDDRSU, Outono)   
355563          (PED, RMPDR, EOVRDJOPANVP, Chuva)   
355598                       (RNADRDDRSU, Outono)   
269418               (SOOLA, EOVRDJOPANVP, Chuva)   
269403                (RNADRDDRSU, Outono, RMPDR)   
269332               (SOOLA, EOVRDJOPANVP, Chuva)   
269337                       (RNADRDDRSU, Outono)   
347435                 (PED, EOVRDJOPANVP, Chuva)   
347446                       (RNADRDDRSU, Outono)   
355582                (RNADRDDRSU, Outono, RMPDR)   
355579                 (PED, EOVRDJOPANVP, Chuva)   
269390   (RNADRDDRSU, RMPDR, EOVRDJOPANVP, Chuva)   
269431                            (Outono, SOOLA)   
269411                      (SOOLA, RMPDR, Chuva)   
269410         (RNADRDDRSU, Outono, EOVRDJOPANVP)   
269412                     (Outono, SOOLA, RMPDR)   


In [8]:
# This code only serves to present the data in a shorter form in chapter 7 of the dissertation

# Select main columns
rules_selected = rules_pressão_acumulada[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


#  View the formatted table
print("Regras que envolvem Chuva (Top 20):\n")
print(rules_selected.head(20))

Regras que envolvem Chuva (Top 20):

                                     antecedents  \
269397       (SOOLA, RMPDR, EOVRDJOPANVP, Chuva)   
269424                      (RNADRDDRSU, Outono)   
355563         (PED, RMPDR, EOVRDJOPANVP, Chuva)   
355598                      (RNADRDDRSU, Outono)   
269418              (SOOLA, EOVRDJOPANVP, Chuva)   
269403               (RNADRDDRSU, Outono, RMPDR)   
269332              (SOOLA, EOVRDJOPANVP, Chuva)   
269337                      (RNADRDDRSU, Outono)   
347435                (PED, EOVRDJOPANVP, Chuva)   
347446                      (RNADRDDRSU, Outono)   
355582               (RNADRDDRSU, Outono, RMPDR)   
355579                (PED, EOVRDJOPANVP, Chuva)   
269390  (RNADRDDRSU, RMPDR, EOVRDJOPANVP, Chuva)   
269431                           (Outono, SOOLA)   
269411                     (SOOLA, RMPDR, Chuva)   
269410        (RNADRDDRSU, Outono, EOVRDJOPANVP)   
269412                    (Outono, SOOLA, RMPDR)   
269409         (RNADRDDRSU,

In [ ]:


# Filter rules involving 'Feriado'
rules_holiday = rules[(rules['antecedents'].apply(lambda x: 'Feriado' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Feriado' in str(x)))]

print("Regras que envolvem 'Feriado':\n", rules_holiday)

In [ ]:
# Filter rules involving 'Fim_de_Semana'
rules_pressão_acumulada = rules[(rules['antecedents'].apply(lambda x: 'Fim_de_Semana' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Fim_de_Semana' in str(x)))]

print("Fim_de_Semana':\n", rules_pressão_acumulada)


In [ ]:
seasons = ['Inverno', 'Verao', 'Primavera', 'Outono']
rules_seasons = rules[rules['antecedents'].apply(lambda x: any(season in str(x) for season in seasons)) |
                      rules['consequents'].apply(lambda x: any(season in str(x) for season in seasons))]

print("Regras que envolvem estações:\n", rules_seasons.head(20))